# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

- Framework는 프로그램 작성시 골격의 역할을 하며, 개발자는 프레임워크의 규칙에 따라 코드를 짜야 한다. (예시: Flask, Django 등)
- Library는 미리 짜둔 코드의 모음집으로, 개발자가 코딩 시 가져와 자유롭게 사용 및 편집 가능하다. (예시: numpy, pandas 등)

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

딥러닝은 머신러닝에 포함되는 개념으로, 머신러닝보다 복잡한 비정형 데이터 혹은 대규모 데이터를 다루는데 딥러닝을 주로 사용한다. 머신러닝과 딥러닝의 큰 차이점은 사람의 개입 여부이다. 머신러닝은 사람이 직접 feature를 지정해주는 대신 딥러닝은 오직 데이터으로부터 문제 해결에 도움이 되는 feature를 스스로 찾아낸다.

### Q3. 아래의 코드에 주석 달기.

In [1]:
# pytorch 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

# GPU 정상 인식 시, 'cuda' or 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 시드 고정
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

cuda is available


In [2]:
# 하이퍼파라미터 설정 (학습률, 배치사이즈, 클래스 수, 에폭 수)

learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [3]:
# MNIST 훈련 데이터셋, 테스트 데이터셋 가져오기

train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST\MNIST\raw\train-images-idx3-ubyte.gz to ./data/MNIST\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST\MNIST\raw\train-labels-idx1-ubyte.gz to ./data/MNIST\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data/MNIST\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/MNIST\MNIST\raw



In [4]:
# 배치 관리를 위한 pytorch의 데이터로더 객체 생성
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

In [5]:
# 커스텀 모델 클래스 작성
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()
        # convolution layer 추가
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        # dropout
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) 
        # maxpooling
        self.mp = nn.MaxPool2d(2)
        # fully connected layer 추가
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 


  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x)

In [6]:
# 모델은 사전에 설정한 device에 할당
model = ConvNet().to(device) 
# crossentropy를 loss 함수로 설정
criterion = nn.CrossEntropyLoss().to(device)
# Adam을 optimizer로 설정
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [7]:
# 설정한 에폭만큼 학습 진행
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        hypothesis = model(data)
        cost = criterion(hypothesis, target)    # loss 함수로 cost 계산
        cost.backward()                         # backpropagation
        optimizer.step()                        # optimizer에 따라 가중치 업데이트
        avg_cost += cost / len(train_loader)    # 평균 cost 계산
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

C:\Users\seenw\anaconda3\envs\aidacon\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch:    1]  cost = 0.331805944
[Epoch:    2]  cost = 0.111243695
[Epoch:    3]  cost = 0.0862488374
[Epoch:    4]  cost = 0.0735230818
[Epoch:    5]  cost = 0.066812925


In [8]:
# 학습한 모델 평가
model.eval()               # 평가 모드로 전환 - evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키는 역할
with torch.no_grad():     # tensor들이 requires_grad=False 상태가 되어, 메모리 사용량을 아껴준다.
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

C:\Users\seenw\anaconda3\envs\aidacon\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Test Accuracy:  98.61 %


## 첫 정규세션 들으시느라 고생 많으셨습니다.